In [ ]:
# Importamos las librerías necesarias para el análisis de datos.
import pandas as pd           # Para manejar y analizar datos en DataFrames.
import numpy as np            # Para operaciones numéricas avanzadas.
import matplotlib.pyplot as plt # Para crear gráficos y visualizaciones.
import seaborn as sns         # Para gráficos estadísticos más atractivos.
import os                     # Para interactuar con el sistema operativo (aunque Path lo sustituye en parte).
from pathlib import Path      # Para manejar rutas de archivos de forma moderna.

In [ ]:
class Config:
    # Clase que almacena la configuración y constantes del proyecto.
    
    # Rutas de entrada y salida de archivos.
    CSV_IN = Path('../db/export.csv')
    CSV_OUT = Path('../data/dataset_enriquecido.csv')
    GRAFICOS_DIR = Path('../docs/graficos/')

    # Mapeo para renombrar columnas con nombres cortos y uniformes.
    COLUMN_MAP = {
        'incapermaparciar_at': 'inc_at',       # Incapacidad Temporal - Accidente de Trabajo (AT)
        'incapermaparciar_el': 'inc_el',       # Incapacidad Temporal - Enfermedad Laboral (EL)
        'muertes_repor_at': 'muertes',         # Muertes reportadas
        'nuevapensioinva_r_at': 'pen_at',      # Nuevas pensiones por AT
        'nuevapensioinva_r_el': 'pen_el',      # Nuevas pensiones por EL
        'presuaccidetrasuce': 'presuntos',     # Presuntos accidentes/sucesos
        'rela_dep': 'dep',                     # Relacionados Dependientes
        'rela_indep': 'indep',                 # Relacionados Independientes
        'a_o_de_informe': 'anio',              # Año de informe
        'mes_de_informe': 'mes'                # Mes de informe
    }
    
    # Mapeo para convertir nombres de meses (en texto) a su número correspondiente.
    MESES_MAP = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }

# Configura el estilo de los gráficos de Seaborn y el tamaño predeterminado de las figuras.
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
def cargar_datos(ruta):
    # Carga el archivo CSV en un DataFrame de Pandas, leyendo todas las columnas como texto (dtype=str).
    try:
        df = pd.read_csv(ruta, dtype=str)
        print(f"Dataset cargado: {len(df)} registros")
        return df
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en {ruta}")
        return pd.DataFrame()

def limpiar_nombres_columnas(df):
    # Limpia los nombres de las columnas: las convierte a minúsculas, elimina tildes,
    # reemplaza caracteres especiales y espacios por guiones bajos.
    df.columns = (
        df.columns.str.lower()
        .str.strip()
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace(r'[^a-z0-9]+', '_', regex=True)
        .str.strip('_')
    )
    print(f"Columnas normalizadas: {list(df.columns)}")
    return df

def renombrar_columnas(df, mapeo):
    # Renombra las columnas del DataFrame usando el diccionario de mapeo definido en Config.
    cols_existentes = {k: v for k, v in mapeo.items() if k in df.columns}
    df.rename(columns=cols_existentes, inplace=True)
    print(f"Columnas renombradas: {list(cols_existentes.values())}")
    return df

def eliminar_duplicados(df):
    # Busca y elimina filas duplicadas del DataFrame.
    inicial = len(df)
    df.drop_duplicates(inplace=True)
    eliminados = inicial - len(df)
    print(f"Duplicados eliminados: {eliminados}")
    return df

def manejar_nulos(df, cols_clave):
    # Elimina filas donde haya valores nulos (NaN) en las columnas clave especificadas.
    inicial = len(df)
    df.dropna(subset=cols_clave, inplace=True)
    eliminados = inicial - len(df)
    print(f"Registros con nulos eliminados: {eliminados}")
    return df

def convertir_numericas(df, cols):
    # Convierte las columnas especificadas a números enteros.
    # Los errores (valores no numéricos) se convierten a NaN, y luego a 0.
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    print(f"Columnas convertidas a numéricas: {cols}")
    return df

In [ ]:
def crear_columna_fecha(df, meses_map):
    # Crea una columna de tipo fecha a partir de las columnas 'anio' y 'mes'.
    try:
        # Convierte el nombre del mes a número. Si es numérico, lo extrae. Si no, usa 1 por defecto.
        df['mes_num'] = df['mes'].astype(str).str.lower().map(meses_map)
        df['mes_num'] = df['mes_num'].fillna(df['mes'].astype(str).str.extract(r'(\d+)')[0].astype(float))
        df['mes_num'] = df['mes_num'].fillna(1).astype(int)
        
        # Combina 'anio', 'mes_num' y un día fijo (15) para crear el objeto de fecha.
        df['fecha'] = pd.to_datetime(
            df['anio'].astype(str) + '-' + 
            df['mes_num'].astype(str) + '-15',
            errors='coerce' # Si hay error en la conversión, pone NaT (Not a Time)
        )
        
        # Elimina las filas donde no se pudo crear una fecha válida.
        df.dropna(subset=['fecha'], inplace=True)
        print(f"Columna 'fecha' creada: {len(df)} registros válidos")
        
    except Exception as e:
        print(f"Error al crear fecha: {e}")
        df['fecha'] = pd.to_datetime('2024-01-15') # Asigna una fecha por defecto en caso de error grave.
    
    return df

def derivar_columnas_temporales(df):
    # Extrae el año, mes, día y trimestre de la columna 'fecha'.
    df['anio'] = df['fecha'].dt.year
    df['mes_num'] = df['fecha'].dt.month
    df['dia'] = df['fecha'].dt.day
    df['trimestre'] = df['fecha'].dt.quarter
    print(f"Columnas temporales derivadas")
    return df

In [ ]:
def configurar_directorio_graficos(directorio):
    # Se asegura de que la carpeta donde se guardarán los gráficos exista.
    directorio.mkdir(parents=True, exist_ok=True)
    print(f"Directorio de gráficos: {directorio}")

def grafico_top_actividades(df, graficos_dir):
    # Gráfico 1: Muestra las 10 actividades económicas con más incidentes de trabajo ('inc_at').
    if 'activec' not in df.columns or 'inc_at' not in df.columns:
        print("Columnas necesarias no encontradas para gráfico de actividades")
        return
    
    # Agrupa y selecciona las 10 principales.
    df_act = df.groupby('activec')['inc_at'].sum().nlargest(10).reset_index()
    
    plt.figure(figsize=(12, 6))
    sns.barplot(x='inc_at', y='activec', data=df_act, palette='viridis') # Crea un gráfico de barras.
    plt.title('Top 10 Actividades Económicas con Mayor Incidentalidad', fontsize=14, fontweight='bold')
    plt.xlabel('Total de Incidentes de Trabajo')
    plt.ylabel('Actividad Económica')
    plt.tight_layout()
    plt.savefig(graficos_dir / '01_incidentes_por_actividad.png', dpi=300, bbox_inches='tight') # Guarda el gráfico.
    plt.close()
    print("Gráfico 1: Incidentes por actividad") # 

def grafico_distribucion_incidentes(df, graficos_dir):
    # Gráfico 2: Muestra la distribución de la cantidad de incidentes ('inc_at') usando un histograma.
    if 'inc_at' not in df.columns:
        print("Columna 'inc_at' no encontrada")
        return
    
    plt.figure(figsize=(10, 6))
    sns.histplot(df['inc_at'], bins=50, kde=True, color='skyblue')
    plt.title('Distribución de Incidentes de Trabajo', fontsize=14, fontweight='bold')
    plt.xlabel('Número de Incidentes')
    plt.ylabel('Frecuencia')
    plt.yscale('log') # Escala logarítmica en el eje Y para mejor visualización.
    plt.tight_layout()
    plt.savefig(graficos_dir / '02_histograma_inc_at.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Gráfico 2: Distribución de incidentes") # 

def grafico_muertes_vs_cobertura(df, graficos_dir):
    # Gráfico 3: Muestra la relación entre el número de trabajadores dependientes ('dep') y las muertes ('muertes').
    if 'dep' not in df.columns or 'muertes' not in df.columns:
        print("Columnas necesarias no encontradas para gráfico muertes vs cobertura")
        return
    
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='dep', y='muertes', data=df, alpha=0.5, color='red', s=30) # Gráfico de dispersión (Scatter plot).
    plt.title('Relación: Trabajadores Dependientes vs Muertes', fontsize=14, fontweight='bold')
    plt.xlabel('Trabajadores Dependientes')
    plt.ylabel('Número de Muertes')
    plt.tight_layout()
    plt.savefig(graficos_dir / '03_muertes_vs_cobertura.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Gráfico 3: Muertes vs cobertura") # 

def grafico_tendencia_anual(df, graficos_dir):
    # Gráfico 4: Muestra la tendencia anual de incidentes y muertes a lo largo del tiempo.
    if 'anio' not in df.columns or 'inc_at' not in df.columns or 'muertes' not in df.columns:
        print("Columnas necesarias no encontradas para gráfico de tendencia")
        return
    
    # Agrupa los datos por año y suma los totales.
    df_anual = df.groupby('anio')[['inc_at', 'muertes']].sum().reset_index()
    
    plt.figure(figsize=(10, 6))
    # Crea un gráfico de líneas para incidentes y otro para muertes.
    plt.plot(df_anual['anio'], df_anual['inc_at'], marker='o', label='Incidentes', linewidth=2)
    plt.plot(df_anual['anio'], df_anual['muertes'], marker='s', label='Muertes', color='red', linewidth=2)
    plt.title('Tendencia Anual: Incidentes y Muertes', fontsize=14, fontweight='bold')
    plt.xlabel('Año')
    plt.ylabel('Total')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(graficos_dir / '04_tendencia_anual.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("Gráfico 4: Tendencia anual") # 

def generar_estadisticas(df, cols):
    # Imprime las estadísticas descriptivas (conteo, media, desviación estándar, etc.)
    # para las columnas numéricas clave.
    cols_existentes = [c for c in cols if c in df.columns]
    if cols_existentes:
        print("\n" + "="*70)
        print("ESTADÍSTICAS DESCRIPTIVAS")
        print("="*70)
        print(df[cols_existentes].describe().transpose())
    else:
        print("No se encontraron columnas para estadísticas")

In [ ]:
def pipeline_completo():
    # Función principal que coordina todas las etapas del procesamiento de datos (pipeline).
    print("\n" + "="*70)
    print("INICIANDO PIPELINE DE PROCESAMIENTO")
    print("="*70 + "\n")
    
    # 1. CARGA
    print("[ 1/4 ] CARGA DE DATOS")
    df = cargar_datos(Config.CSV_IN)
    if df.empty:
        return
    
    # 2. LIMPIEZA
    print("\n[ 2/4 ] LIMPIEZA DE DATOS")
    df = limpiar_nombres_columnas(df)
    df = renombrar_columnas(df, Config.COLUMN_MAP)
    df = eliminar_duplicados(df)
    df = manejar_nulos(df, ['activec', 'dpto']) # Elimina nulos en Actividad Económica y Departamento.
    
    # Columnas que se convertirán a números.
    cols_numericas = ['inc_at', 'inc_el', 'muertes', 'pen_at', 'pen_el', 'presuntos', 'dep', 'indep']
    df = convertir_numericas(df, cols_numericas)
    
    # 3. ENRIQUECIMIENTO
    print("\n[ 3/4 ] ENRIQUECIMIENTO DE DATOS")
    df = crear_columna_fecha(df, Config.MESES_MAP)
    df = derivar_columnas_temporales(df)
    
    # Guardar dataset procesado en un nuevo archivo CSV.
    Config.CSV_OUT.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(Config.CSV_OUT, index=False, encoding='utf-8')
    print(f"Dataset enriquecido guardado: {Config.CSV_OUT}")
    
    # 4. VISUALIZACIÓN Y EDA (Análisis Exploratorio de Datos)
    print("\n[ 4/4 ] ANÁLISIS EXPLORATORIO Y VISUALIZACIÓN")
    configurar_directorio_graficos(Config.GRAFICOS_DIR)
    
    # Genera estadísticas y todos los gráficos.
    generar_estadisticas(df, cols_numericas)
    grafico_top_actividades(df, Config.GRAFICOS_DIR)
    grafico_distribucion_incidentes(df, Config.GRAFICOS_DIR)
    grafico_muertes_vs_cobertura(df, Config.GRAFICOS_DIR)
    grafico_tendencia_anual(df, Config.GRAFICOS_DIR)
    
    # Mensaje de finalización.
    print("\n" + "="*70)
    print("PIPELINE COMPLETADO EXITOSAMENTE")
    print("="*70)
    print(f"\nRegistros finales: {len(df)}")
    print(f"Datos: {Config.CSV_OUT}")
    print(f"Gráficos: {Config.GRAFICOS_DIR}")

In [ ]:
if __name__ == "__main__":
    # Inicia la ejecución del pipeline completo.
    pipeline_completo()


INICIANDO PIPELINE DE PROCESAMIENTO

[ 1/4 ] CARGA DE DATOS
Dataset cargado: 88620 registros

[ 2/4 ] LIMPIEZA DE DATOS
Columnas normalizadas: ['dpto', 'mpio', 'codigo_de_la_arl', 'ao_de_informe', 'mes_de_informe', 'activec', 'rela_dep', 'rela_indep', 'presuaccidetrasuce', 'muertes_repor_at', 'nuevapensioinva_r_at', 'nuevapensioinva_r_el', 'incapermaparciar_at', 'incapermaparciar_el']
Columnas renombradas: ['inc_at', 'inc_el', 'muertes', 'pen_at', 'pen_el', 'presuntos', 'dep', 'indep', 'mes']
Duplicados eliminados: 0
Registros con nulos eliminados: 0
Columnas convertidas a numéricas: ['inc_at', 'inc_el', 'muertes', 'pen_at', 'pen_el', 'presuntos', 'dep', 'indep']

[ 3/4 ] ENRIQUECIMIENTO DE DATOS
Error al crear fecha: 'anio'
Columnas temporales derivadas
Dataset enriquecido guardado: ..\data\dataset_enriquecido.csv

[ 4/4 ] ANÁLISIS EXPLORATORIO Y VISUALIZACIÓN
Directorio de gráficos: ..\docs\graficos

ESTADÍSTICAS DESCRIPTIVAS
             count       mean         std  min  25%  50% 

C:\Users\aplazas\AppData\Local\Temp\ipykernel_25088\2435080152.py:15: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='inc_at', y='activec', data=df_act, palette='viridis')


Gráfico 1: Incidentes por actividad
Gráfico 2: Distribución de incidentes
Gráfico 3: Muertes vs cobertura
Gráfico 4: Tendencia anual

PIPELINE COMPLETADO EXITOSAMENTE

Registros finales: 88620
Datos: ..\data\dataset_enriquecido.csv
Gráficos: ..\docs\graficos
